In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import DataFrame 
import nltk

from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import LogisticRegression
from sklearn import neighbors
from scipy.spatial.distance import cosine
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

import re
import string
from sklearn.metrics import mean_squared_error

In [2]:
df= pd.read_json("Automotive_5.json",lines = True)

In [3]:
df.head()

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,B00002243X,"[4, 4]",5,I needed a set of jumper cables for my new car...,"08 17, 2011",A3F73SC1LY51OO,Alan Montgomery,Work Well - Should Have Bought Longer Ones,1313539200
1,B00002243X,"[1, 1]",4,"These long cables work fine for my truck, but ...","09 4, 2011",A20S66SKYXULG2,alphonse,Okay long cables,1315094400
2,B00002243X,"[0, 0]",5,Can't comment much on these since they have no...,"07 25, 2013",A2I8LFSN2IS5EO,Chris,Looks and feels heavy Duty,1374710400
3,B00002243X,"[19, 19]",5,I absolutley love Amazon!!! For the price of ...,"12 21, 2010",A3GT2EWQSO45ZG,DeusEx,Excellent choice for Jumper Cables!!!,1292889600
4,B00002243X,"[0, 0]",5,I purchased the 12' feet long cable set and th...,"07 4, 2012",A3ESWJPAVRPWB4,E. Hernandez,"Excellent, High Quality Starter Cables",1341360000


In [4]:
df = df[['asin','overall','reviewerID']].copy()

In [5]:
df.head()

,asin,overall,reviewerID
0,B00002243X,5,A3F73SC1LY51OO
1,B00002243X,4,A20S66SKYXULG2
2,B00002243X,5,A2I8LFSN2IS5EO
3,B00002243X,5,A3GT2EWQSO45ZG
4,B00002243X,5,A3ESWJPAVRPWB4


In [6]:
from sklearn.model_selection import train_test_split
# Splitting train and test data at the start
df_train, df_test = train_test_split(df,test_size=0.2,stratify = df['reviewerID'])

In [7]:
df_train.head()

,asin,overall,reviewerID
19883,B00BQYCIVI,4,A2UAT3A0KEON7L
3642,B0009U7Y2I,5,A6II57Q2ZA9M7
474,B00029WYDU,4,A1N087XH2AUKWP
13183,B002BN5ZT0,5,A23PYCN8O9IUNH
20411,B00HJ6OBUI,4,A3U1WPDQLP9CQ


In [8]:
df_train = df_train.sort_index(axis=0)

In [9]:
df_train.head()

,asin,overall,reviewerID
1,B00002243X,4,A20S66SKYXULG2
2,B00002243X,5,A2I8LFSN2IS5EO
3,B00002243X,5,A3GT2EWQSO45ZG
4,B00002243X,5,A3ESWJPAVRPWB4
5,B00002243X,5,A1ORODEBRN64C


In [10]:
df_original_train = df_train.copy()
df_train = df_train.pivot(index = "asin" , columns = "reviewerID" , values = "overall")
df_train = df_train.sort_index(axis=1)

In [11]:
df_train.head()

reviewerID,A00473363TJ8YSZ3YAGG9,A0473259F6GQNBD88IYN,A09567722SXKYZTV7OFCJ,A10063PJ5C9WQQ,A100UD67AHFODS,A100WO06OQR8BQ,A1027BOR2MECKT,A105S56ODHGJEK,A108AWE1CYYZVB,A109KHEJL08HW9,...,AZD412AH3W49I,AZFODJ6KW6KQH,AZJPNK73JF3XP,AZOK5STV85FBJ,AZP8DAHJCFWPH,AZPZROZJIKBE8,AZT5UV3ZRB4TT,AZTAK8PFB0NLV,AZV2U6GU5QA6C,AZYBZEVISD0YX
asin,,,,,,,,,,,,,,,,,,,,,
B00002243X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B00002243Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B00008BKX5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B00008RW9U,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B00008RW9V,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
X = df_train.values
Y = df_train.sum(axis = 1)
Z =df_train.count(axis =1).astype('float64')
Y = Y.div(Z,axis=0)
cosine
df_new_train = df_train.sub(Y,axis=0).fillna(0)

In [13]:
df_new_train.head()

reviewerID,A00473363TJ8YSZ3YAGG9,A0473259F6GQNBD88IYN,A09567722SXKYZTV7OFCJ,A10063PJ5C9WQQ,A100UD67AHFODS,A100WO06OQR8BQ,A1027BOR2MECKT,A105S56ODHGJEK,A108AWE1CYYZVB,A109KHEJL08HW9,...,AZD412AH3W49I,AZFODJ6KW6KQH,AZJPNK73JF3XP,AZOK5STV85FBJ,AZP8DAHJCFWPH,AZPZROZJIKBE8,AZT5UV3ZRB4TT,AZTAK8PFB0NLV,AZV2U6GU5QA6C,AZYBZEVISD0YX
asin,,,,,,,,,,,,,,,,,,,,,
B00002243X,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00002243Z,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00008BKX5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00008RW9U,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00008RW9V,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
nbrs_train = NearestNeighbors(n_neighbors= 10 , algorithm='auto',metric ='cosine').fit(df_new_train.values)
distances, indices = nbrs_train.kneighbors(df_new_train,return_distance=True)

In [15]:
distances

array([[0.00000000e+00, 6.57673402e-01, 9.06066356e-01, ...,
        9.41074435e-01, 9.45288986e-01, 9.48351958e-01],
       [2.22044605e-16, 7.42429958e-01, 8.33333333e-01, ...,
        9.37005921e-01, 9.37005921e-01, 9.39514162e-01],
       [0.00000000e+00, 8.81724597e-01, 8.89903623e-01, ...,
        9.56356422e-01, 9.71994398e-01, 9.72426904e-01],
       ...,
       [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
       [2.22044605e-16, 7.31971866e-01, 7.95875855e-01, ...,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
       [0.00000000e+00, 7.39759981e-01, 8.14445253e-01, ...,
        9.43210972e-01, 9.49604737e-01, 9.50807306e-01]])

In [16]:
indices

array([[   0,   67,  937, ...,  156,  734,  708],
       [   1, 1088, 1750, ..., 1492,  697,  703],
       [   2,  209, 1373, ...,   51,  412, 1541],
       ...,
       [1224, 1225, 1220, ..., 1229, 1227, 1217],
       [1832,  344,  811, ..., 1223, 1227, 1224],
       [1833, 1603, 1377, ..., 1669,  772,  613]])

In [17]:
asin_list = df_new_train.index.values.tolist()
index_list = indices.tolist()

In [18]:
for i in range(5):
    string = str(asin_list[i]) + " : "
    for x in index_list[i][1:]:
            string += asin_list[x] + " , "
    print(string)

B00002243X : B0002KRABU , B001D8XA50 , B0018PSASU , B000EJS9IM , B000C5GAQQ , B000HBGHB8 , B00042JSNY , B000P17NXQ , B000NJ6EZ8 , 
B00002243Z : B001V8U12M , B00AOIM2CC , B000JFJLQA , B002QAC66I , B0002M9QK6 , B00042KJQ4 , B004P91FCO , B000MY3ML8 , B000NCOKV0 , 
B00008BKX5 : B000760FWU , B003VAYJAS , B000O80B5M , B000BXKYYI , B000RN04RO , B000EVYGZA , B0002H335A , B000BZG29W , B005ASY23I , 
B00008RW9U : B00167Y3ZO , B001MXTOKQ , B003UT3S6Q , B0033A5PPO , B00D8DR0AO , B005KDOA9Y , B000E9TS6O , B0000AY3X0 , B0015PI7A4 , 
B00008RW9V : B002WQHY9K , B002UMC1HQ , B002UX1T78 , B002VVC4TG , B002VVIC06 , B002UD0GGS , B002XL2IEA , B002X6VXL4 , B002UD0GFE , 


In [19]:
item_avg = df_original_train.groupby(['asin']).mean()
user_avg = df_original_train.groupby(['reviewerID']).mean()
mean_avg = df_original_train["overall"].mean()
item_map = df_new_train.index.values

In [20]:
item_avg.head()

,overall
asin,
B00002243X,4.750000
B00002243Z,4.666667
B00008BKX5,4.333333
B00008RW9U,4.466667
B00008RW9V,5.000000


In [21]:
df_asin = df.groupby('asin').count()
df_asin = df_asin.index.values

In [22]:
df_asin

array(['B00002243X', 'B00002243Z', 'B00008BKX5', ..., 'B00JGM7Z5Q',
       'B00JGP8I1S', 'B00KIAQ8VW'], dtype=object)

In [23]:
df_reviewerID = df.groupby('reviewerID').count()
df_reviewerID = df_reviewerID.index.values

In [24]:
df_reviewerID

array(['A00473363TJ8YSZ3YAGG9', 'A0473259F6GQNBD88IYN',
       'A09567722SXKYZTV7OFCJ', ..., 'AZTAK8PFB0NLV', 'AZV2U6GU5QA6C',
       'AZYBZEVISD0YX'], dtype=object)

In [25]:
df_dict = {}
for line in df_original_train.values:
    if line[0] in df_dict:
        df_dict[line[0]].append(line[2])
    else:
        df_dict[line[0]] = [line[2]]

In [26]:
predict_for_this = []
count = 0
for i in range(len(df_reviewerID)):
    for j in range(len(df_asin)):
#         print(df_asin[i])
        if df_asin[j] not  in  df_dict or not df_reviewerID[i] in df_dict[df_asin[j]]:
            predict_for_this.append((df_reviewerID[i],df_asin[j]))

In [31]:
final_prediction = {}

for i in range(10000):
#__________________________________________________________________________________________________________
# for i in range(len(predict_for_this)): This loop needs to be run instead of above one. 5352407 records: takes time
#__________________________________________________________________________________________________________
    
    item_asin = predict_for_this[i][1]
#     print(item_asin)
    item_userID = predict_for_this[i][0]
#     print(item_asin,item_userID)
    
    predicted_rating = 0
    if item_asin in item_avg.index:
        bi = item_avg.loc[item_asin]['overall'] - mean_avg
    else:
        bi = 0
    if item_userID in user_avg.index:
        bx = user_avg.loc[item_userID]['overall'] - mean_avg
    else:
        bx = 0
            
    baseline_estimate = mean_avg + bi + bx
    predicted_rating = baseline_estimate
#     print(baseline_estimate)
    
    numerator = 0 
    denomenator = 0
    
    if bi != 0 :
        
        item  = np.where(item_map == item_asin)[0][0]
        item_nbrs = indices[item:item+1]
        dist_nbrs = distances[item:item+1]
        cnt = 0;
        k = 0
        
        for j in item_nbrs:
            temp = df_original_train.loc[(df_original_train['asin'] == item_map[j[0]]) &  (df_original_train['reviewerID'] == item_userID)]
            if(temp.size != 0):
                dist = dist_nbrs[cnt:cnt+1]
                numerator += ((1 - dist[0][k])*(temp['overall'].values[0] - (item_avg.loc[temp['asin']]['overall'] - mean_avg) ))

                denomenator += (1 - dist[0][k])
            cnt = cnt + 1
            k+= 1
        if denomenator:
            predicted_rating += numerator/denomenator 

    if item_userID in final_prediction:
        final_prediction[item_userID].append((item_asin,predicted_rating))
    else:
        final_prediction[(item_userID)] = [(item_asin,predicted_rating)]

#___________________________________________________________
#        Sort final_prediction on basis of rating
#___________________________________________________________

In [28]:
final_prediction

{'A00473363TJ8YSZ3YAGG9': [('B00002243X', 4.527048479667847),
  ('B00002243Z', 4.443715146334514),
  ('B00008BKX5', 4.11038181300118),
  ('B00008RW9U', 4.243715146334514),
  ('B00008RW9V', 4.777048479667847),
  ('B00009RAX7', 4.527048479667847),
  ('B00009W3G7', 4.477048479667848),
  ('B0000ATZD5', 4.348477051096419),
  ('B0000ATZDE', 4.177048479667848),
  ('B0000AXNMO', 4.464548479667847),
  ('B0000AXPDU', 3.5770484796678472),
  ('B0000AXRH5', 4.538953241572609),
  ('B0000AXTUY', 4.777048479667847),
  ('B0000AXU02', 4.777048479667847),
  ('B0000AXY62', 4.205619908239276),
  ('B0000AY3X0', 4.443715146334514),
  ('B0000AY60S', 4.277048479667847),
  ('B0000AY9W6', 4.205619908239276),
  ('B0000AYES4', 4.777048479667847),
  ('B0000AZ9KS', 3.7770484796678474),
  ('B0000DYVN9', 4.062762765382133),
  ('B0000UUX34', 4.777048479667847),
  ('B000182DPG', 4.577048479667847),
  ('B0001L0DFA', 4.152048479667847),
  ('B00020CATC', 3.9770484796678476),
  ('B00020CAUG', 3.877048479667847),
  ('B00020C

In [34]:
for x in final_prediction:
    print(len(final_prediction[x]))

1831
1826
1831
1831
1829
852


In [35]:
for x in final_prediction:
    final_prediction[x] = sorted(final_prediction[x],key = lambda x:x[1],reverse= True)[:10]

In [36]:
final_prediction

{'A00473363TJ8YSZ3YAGG9': [('B00008RW9V', 4.777048479667847),
  ('B0000AXTUY', 4.777048479667847),
  ('B0000AXU02', 4.777048479667847),
  ('B0000AYES4', 4.777048479667847),
  ('B0000UUX34', 4.777048479667847),
  ('B00020CB2S', 4.777048479667847),
  ('B000221M94', 4.777048479667847),
  ('B00029WRJQ', 4.777048479667847),
  ('B00029WY9Y', 4.777048479667847),
  ('B0002CSQ7A', 4.777048479667847)],
 'A0473259F6GQNBD88IYN': [('B00008RW9V', 5.415937368556737),
  ('B0000AXTUY', 5.415937368556737),
  ('B0000AXU02', 5.415937368556737),
  ('B0000AYES4', 5.415937368556737),
  ('B0000UUX34', 5.415937368556737),
  ('B00020CB2S', 5.415937368556737),
  ('B000221M94', 5.415937368556737),
  ('B00029WRJQ', 5.415937368556737),
  ('B00029WY9Y', 5.415937368556737),
  ('B0002CSQ7A', 5.415937368556737)],
 'A09567722SXKYZTV7OFCJ': [('B00008RW9V', 5.527048479667847),
  ('B0000AXTUY', 5.527048479667847),
  ('B0000AXU02', 5.527048479667847),
  ('B0000AYES4', 5.527048479667847),
  ('B0000UUX34', 5.527048479667847),